# 1. 훈련 데이터 전처리하기
## <font color = "purple">문자 시퀀스 apple을 입력받으면 pple!를 출력하는 RNN을 구현합니다. </font>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

In [2]:
input_str = "apple"
label_str = "pple!"
char_vocab = sorted(list(set(input_str + label_str)))
vocab_size = len(char_vocab)
print("문자 집합의 크기 :", vocab_size)

문자 집합의 크기 : 5


In [3]:
input_size = vocab_size
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [4]:
# 문자 집합에 고유 정수를 부여합니다
char_to_idx = {word : i for i, word in enumerate(char_vocab)}
print(char_to_idx)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [5]:
# 정수로부터 문자를 얻을 수 있는 index_to_char를 만듭니다
index_to_char = {}
for key, value in char_to_idx.items():
    index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [6]:
x_data = [char_to_idx[c] for c in input_str]
y_data = [char_to_idx[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [7]:
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [8]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [9]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print(X)
print(Y)

tensor([[[0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.]]])
tensor([[4, 4, 3, 2, 0]])


In [10]:
print("훈련 데이터의 크기 :", X.shape)
print("레이블의 크기:", Y.shape)

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기: torch.Size([1, 5])


# 2. 모델 구현하기

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [34]:
input_size = vocab_size # vocab_size = 5
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [35]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first = True)
        self.fc = nn.Linear(hidden_size, output_size, bias=True)
        
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [36]:
net = Net(input_size, hidden_size, output_size)

In [37]:
outputs = net(X)
print(outputs.shape)

torch.Size([1, 5, 5])


In [38]:
# 나중에 정확도를 측정할 떄는 배치 차원과 time-step차원을 하나로 합친다.
print(outputs.view(-1, input_size).shape)

torch.Size([5, 5])


In [39]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


In [40]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [41]:
for i in range(101):
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
        #아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드
        result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
        result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
        print("epoch :", i, "loss :", loss, "prediction: ", result, "true Y:", y_data, "prediction str:", result_str)

epoch : 0 loss : tensor(1.5849, grad_fn=<NllLossBackward>) prediction:  [[4 2 2 2 2]] true Y: [[4, 4, 3, 2, 0]] prediction str: peeee
epoch : 10 loss : tensor(0.0579, grad_fn=<NllLossBackward>) prediction:  [[4 4 3 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str: pple!
epoch : 20 loss : tensor(0.0047, grad_fn=<NllLossBackward>) prediction:  [[4 4 3 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str: pple!
epoch : 30 loss : tensor(0.0016, grad_fn=<NllLossBackward>) prediction:  [[4 4 3 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str: pple!
epoch : 40 loss : tensor(0.0010, grad_fn=<NllLossBackward>) prediction:  [[4 4 3 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str: pple!
epoch : 50 loss : tensor(0.0007, grad_fn=<NllLossBackward>) prediction:  [[4 4 3 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str: pple!
epoch : 60 loss : tensor(0.0007, grad_fn=<NllLossBackward>) prediction:  [[4 4 3 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str: pple!
epoch : 70 loss : tensor(0.0006, grad_fn=<NllLossBackwar